In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp


In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose


In [3]:
DATA_PATH = os.path.join('GYM_Data')
exercises = ['curl_up', 'curl_down', 'squat_up', 'squat_down']
no_sequences = 30
sequence_length = 30

for exercise in exercises:
    for sequence in range(no_sequences):
        os.makedirs(os.path.join(DATA_PATH, exercise, str(sequence)), exist_ok=True)


In [5]:
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Pose model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for exercise in exercises:
        for sequence in range(no_sequences):
            # Add a buffer and display the class name being collected
            # for buffer in range(2):
            #     ret, frame = cap.read()
            #     cv2.putText(frame, f'Collecting frames for {exercise} sequence {sequence}. Starting in {2-buffer} seconds...', 
            #                 (15, 25), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
            #     cv2.imshow('Webcam Feed', frame)
            #     cv2.waitKey(1000)  # Wait for 1 second

            start_time = cv2.getTickCount()
            for frame_num in range(sequence_length):
                # Capture frames based on the time elapsed
                while cv2.getTickCount() - start_time < (frame_num + 1) * cv2.getTickFrequency() / (sequence_length / 3):
                    pass

                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, pose)

                # Draw the pose annotations on the image
                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                keypoints = extract_keypoints(results)

                # Save keypoints as a numpy file
                npy_path = os.path.join(DATA_PATH, exercise, str(sequence), f"{frame_num}.npy")
                np.save(npy_path, keypoints)

                # Display the collecting status
                cv2.putText(image, f'Collecting {exercise}, Seq {sequence}, Frame {frame_num}', 
                            (15, 25), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.imshow('Webcam Feed', image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

